# Download data from google drive

In [ ]:
import src.gdrive_uploader as d
import src.markdown_writer as mw

import pandas as pd


d.get_data() # pay attention - this operation take a lot of time! If 
            # OS fall in sleep mode operation will be stoped!


In [2]:
#Upload data
df_lectures = pd.read_csv('./data/lectures.csv')
df_questions = pd.read_csv('./data/questions.csv')
df_train = pd.read_csv('./data/train.csv')

In [ ]:
# Some subset of data
df_all_questions = df_train.loc[df_train['content_type_id']==0]
df_all_lectures = df_train.loc[df_train['content_type_id']==1]

# Ordinary data
num_students = len(df_train['user_id'].unique())
num_unique_questions = len(df_questions['question_id'].unique())
num_all_questions = len(df_all_questions['content_id'])
num_lectures = len(df_lectures['lecture_id'].unique())

# User data observation
df_user_correct_answer = df_all_questions[['user_id', 'answered_correctly']].groupby('user_id').sum()
#df_user_time_for_question

most_smart_student = df_user_correct_answer.loc[df_user_correct_answer['answered_correctly'] == df_user_correct_answer['answered_correctly'].max()].index[0]
less_smart_student = df_user_correct_answer.loc[df_user_correct_answer['answered_correctly'] == df_user_correct_answer['answered_correctly'].min()].index[0]


# Questions data observation

df_questions_freq = df_all_questions[['content_id', 'row_id']].groupby('content_id').count()
df_correct_answers_freq = df_all_questions[['content_id', 'answered_correctly']].groupby('content_id').sum()
df_mean = df_questions_freq.join(df_correct_answers_freq)
df_mean['mean'] = df_mean['answered_correctly']/df_mean['row_id']

most_freq_question = df_questions_freq.loc[df_questions_freq['row_id'] == df_questions_freq['row_id'].max()].index[0]
less_freq_question = df_questions_freq.loc[df_questions_freq['row_id'] == df_questions_freq['row_id'].min()].index[0]

most_easy_question = df_correct_answers_freq.loc[df_correct_answers_freq['answered_correctly'] == df_correct_answers_freq['answered_correctly'].max()].index[0]
most_hard_question = df_correct_answers_freq.loc[df_correct_answers_freq['answered_correctly'] == df_correct_answers_freq['answered_correctly'].min()].index[0]

most_easy_question_mean = df_mean.loc[df_mean['mean'] == df_mean['mean'].max()].index[0]
most_hard_question_mean = df_mean.loc[df_mean['mean'] == df_mean['mean'].min()].index[0]

corr_between_freq_answers = df_mean[['row_id', 'answered_correctly']].corr()['row_id']['answered_correctly']

# Create main time distribution analysis
df_mean_time = df_all_questions[['user_id', 'prior_question_elapsed_time']].groupby('user_id').mean()
df_mean_time['prior_question_elapsed_time'] = df_mean_time['prior_question_elapsed_time'].round(2)
df_mean_time_with_answers = df_mean_time.join(df_user_correct_answer)

mean_time = df_all_questions['prior_question_elapsed_time'].dropna().mean()
median_time = df_all_questions['prior_question_elapsed_time'].dropna().median()
mode_time = df_all_questions['prior_question_elapsed_time'].dropna().mode().mean()
std_time = df_all_questions['prior_question_elapsed_time'].dropna().std()

most_fast_user = df_mean_time.loc[df_mean_time['prior_question_elapsed_time'] == df_mean_time['prior_question_elapsed_time'].max()].index[0]
less_fast_user = df_mean_time.loc[(df_mean_time['prior_question_elapsed_time'] == 
                                   df_mean_time['prior_question_elapsed_time'].loc[df_mean_time['prior_question_elapsed_time']!=0].min())].index[0]
corr_of_fastest_answer = df_mean_time_with_answers[['prior_question_elapsed_time', 'answered_correctly']].corr()['answered_correctly']['prior_question_elapsed_time']

# Lectures overview
user_attendance = df_all_lectures[['user_id', 'row_id']].groupby('user_id').count()
mean_attendance = user_attendance['row_id'].mean()
median_attendance = user_attendance['row_id'].dropna().median()
mode_attendance = user_attendance['row_id'].dropna().mode().mean()
std_attendance = user_attendance['row_id'].std()

corr_attendance = df_user_correct_answer.join(user_attendance).corr()['answered_correctly']['row_id']

# Return result in console

In [57]:
# User overview
print('\n'+'\033[4mОбщая информация\033[0m\n')
print('Количество учеников в выборке: ' + str(num_students))
print('Ученик с наибольшим количеством правильных ответов: ' + str(most_smart_student))
print('Ученик с наименьшим количеством правильных ответов: ' + str(less_smart_student))

# Questions overview
print('\n'+'\033[4mБлок информации о вопросах\033[0m\n')
print('Количество уникальных вопросов, которые могли быть заданы студентам: ' + str(num_unique_questions)) # fast code
#len(df_train.loc[df_train['content_type_id']==0]['content_id'].unique())  code for validate 
print('Количество вопросов, которые были заданы всем студентам: ' + str(num_all_questions))
print('Среднее количество вопросов на одного студента: ' + str(round(num_all_questions/num_students)))
print('ID самого частого вопроса: ' + str(most_freq_question))
print('ID самого редкого вопроса: ' + str(less_freq_question))
print('ID самого легкого вопроса: ' + str(most_easy_question))
print('ID самого сложного вопроса: ' + str(most_freq_question))
print('\n\033[36mВывод 1: наиболее сложный вопрос тот, что был в выборке чаще всего\033[0m')
print('ID самого легкого вопроса, с учетом их частоты: '+ str(most_easy_question_mean))
print('ID самого сложного вопроса, с учетом их частоты: '+ str(most_hard_question_mean))
print('\n\033[36mВывод 2: наиболее сложный вопрос в среднем тот, который встречается реже всего\033[0m')
print('\033[32mИз вывода 1 и 2 можно говорить о высокой кореляции между частотой встречаемости вопроса и правильных ответов на него, при этом она положительная\033[0m')
print('\nКоэффициент корреляции между сложностью ответа и его частотой встречаемости: ' + str(round(corr_between_freq_answers, 2)))

# Time overview
print('\n'+'\033[4mБлок информации о времени\033[0m\n')
print('Среднее время ответа: '+str(round(mean_time, 2)))
print('Мода времени ответа: '+str(round(mode_time, 2)))
print('Медианное время ответа: '+str(round(median_time, 2)))
print('СКО времени ответа: '+str(round(std_time, 2)))
print('Корреляция между правильным ответом и его скоростью: '+str(round(corr_of_fastest_answer, 2)))
print('Студент, который давал самые быстрые ответы на вопросы: '+str(most_fast_user))
print('Студент, который давал самые медленные ответы на вопросы: '+str(less_fast_user))
#Lecture overview
print('\n'+'\033[4mБлок информации о лекциях\033[0m\n')
print('Количество лекций, проведенных студентам: ' + str(num_lectures)) # fast code
print('Среднее количество посещений студентами: '+str(round(mean_attendance, 2)))
print('Медианное количество посещений студентами: '+str(round(median_attendance, 2)))
print('Мода количества посещений студентами: '+str(round(mode_attendance, 2)))
print('СКО количества посещений студентами: '+str(round(std_attendance, 2)))
print('Корреляция между посещениями студентами лекций и их правильными ответами: '+\
      str(round(corr_attendance, 2)))
#len(df_train.loc[df_train['content_type_id']==1]['content_id'].unique())  code for validate 





Общая информация

Количество учеников в выборке: 393656
Ученик с наибольшим количеством правильных ответов: 2139561972
Ученик с наименьшим количеством правильных ответов: 1360462

Блок информации о вопросах

Количество уникальных вопросов, которые могли быть заданы студентам: 13523
Количество вопросов, которые были заданы всем студентам: 99271300
Среднее количество вопросов на одного студента: 252
ID самого частого вопроса: 6116
ID самого редкого вопроса: 1484
ID самого легкого вопроса: 7900
ID самого сложного вопроса: 6116

Вывод 1: наиболее сложный вопрос тот, что был в выборке чаще всего
ID самого легкого вопроса, с учетом их частоты: 3572
ID самого сложного вопроса, с учетом их частоты: 1484

Вывод 2: наиболее сложный вопрос в среднем тот, который встречается реже всего
Из вывода 1 и 2 можно говорить о высокой кореляции между частотой встречаемости вопроса и правильных ответов на него, при этом она положительная

Коэффициент корреляции между сложностью ответа и его частотой встреч

# Return result in Markdown file

In [61]:

writer = mw.Markdown_writer('./results/result.md')

# User overview
writer.write_in_file('Общая информация', 'header')
writer.write_in_file('Количество учеников в выборке: ' + str(num_students))
writer.write_in_file('Ученик с наибольшим количеством правильных ответов: ' + str(most_smart_student))
writer.write_in_file('Ученик с наименьшим количеством правильных ответов: ' + str(less_smart_student))
writer.write_in_file('', 'slide')

# Questions overview
writer.write_in_file('Блок информации о вопросах', 'header')
writer.write_in_file('Количество уникальных вопросов, которые могли быть заданы студентам: ' + str(num_unique_questions)) # fast code
#len(df_train.loc[df_train['content_type_id']==0]['content_id'].unique())  code for validate 
writer.write_in_file('Количество вопросов, которые были заданы всем студентам: ' + str(num_all_questions))
writer.write_in_file('Среднее количество вопросов на одного студента: ' + str(round(num_all_questions/num_students)))
writer.write_in_file('ID самого частого вопроса: ' + str(most_freq_question))
writer.write_in_file('ID самого редкого вопроса: ' + str(less_freq_question))
writer.write_in_file('ID самого легкого вопроса: ' + str(most_easy_question))
writer.write_in_file('ID самого сложного вопроса: ' + str(most_freq_question))
writer.write_in_file('Вывод 1: наиболее сложный вопрос тот, что был в выборке чаще всего', 'bold')
writer.write_in_file('ID самого легкого вопроса, с учетом их частоты: '+ str(most_easy_question_mean))
writer.write_in_file('ID самого сложного вопроса, с учетом их частоты: '+ str(most_hard_question_mean))
writer.write_in_file('Вывод 2: наиболее сложный вопрос в среднем тот, который встречается реже всего', 'bold')
writer.write_in_file('Из вывода 1 и 2 можно говорить о высокой кореляции между частотой встречаемости вопроса и правильных ответов на него, при этом она положительная', 'italic_bold')
writer.write_in_file('Коэффициент корреляции между сложностью ответа и его частотой встречаемости: ' + str(round(corr_between_freq_answers, 2)))
writer.write_in_file('', 'slide')

# Time overview
writer.write_in_file('Блок информации о времени', 'header')
writer.write_in_file('Среднее время ответа: '+str(round(mean_time, 2)))
writer.write_in_file('Мода времени ответа: '+str(round(mode_time, 2)))
writer.write_in_file('Медианное время ответа: '+str(round(median_time, 2)))
writer.write_in_file('СКО времени ответа: '+str(round(std_time, 2)))
writer.write_in_file('Корреляция между правильным ответом и его скоростью: '+str(round(corr_of_fastest_answer, 2)))
writer.write_in_file('Студент, который давал самые быстрые ответы на вопросы: '+str(most_fast_user))
writer.write_in_file('Студент, который давал самые медленные ответы на вопросы: '+str(less_fast_user))
writer.write_in_file('', 'slide')

#Lecture overview
writer.write_in_file('Блок информации о лекциях', 'header')
writer.write_in_file('Количество лекций, проведенных студентам: ' + str(num_lectures))
writer.write_in_file('Среднее количество посещений студентами: '+str(round(mean_attendance, 2)))
writer.write_in_file('Медианное количество посещений студентами: '+str(round(median_attendance, 2)))
writer.write_in_file('Мода количества посещений студентами: '+str(round(mode_attendance, 2)))
writer.write_in_file('СКО количества посещений студентами: '+str(round(std_attendance, 2)))
writer.write_in_file('Корреляция между посещениями студентами лекций и их правильными ответами: '+\
      str(round(corr_attendance, 2)))
writer.write_in_file('', 'slide')